In [1]:
#모듈 불러오기 
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import sys
import re
import math
import numpy  
import pandas as pd  
import xlwt
import pyautogui

import random
import os  

from selenium.webdriver.support.ui import Select


#크롤링 기본 셋팅
print("=" *80)
print("연습문제 6-5 : 블로그 크롤러 : 여러건의 네이버 블로그 정보 추출하여 저장하기")
print("=" *80)

key_word = input('''1. 크롤링할 키워드는 무엇입니까?(예:여행):''')

include_word = input('''2. 결과에서 반드시 포함하는 단어를 입력하세요(예:국내, 바닷가)
(여러개일 경우 , 로 구분해서 입력하고 없으면 엔터 입력하세요): ''')

exclude_word = input('''3. 결과에서 제외할 단어를 입력하세요(예:분양권, 해외)
(여러개일 경우 , 로 구분해서 입력하고 없으면 엔터 입력하세요):''')

start_date = input('4. 조회 시작일자 입력(예:2017-01-01):')

end_date = input('5. 조회 종료일자 입력(예:2017-12-31):')

cnt = int(input('6. 크롤링 할 건수는 몇건입니까?(최대 1000건): '))


f_dir = input("7. 파일을 저장할 폴더명만 쓰세요(예:/Users/kibeomkim/Desktop/folder1/): ")
if f_dir=='' :
    f_dir='/Users/kibeomkim/Desktop/웹크롤링_결과_데이터/'
    
    
# 저장될 파일위치와 이름 지정하기
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+key_word)
os.chdir(f_dir+s+'-'+key_word)

ff_name= s+'-'+key_word+'.txt'
fc_name= s+'-'+key_word+'.csv'
fx_name= s+'-'+key_word+'.xls'

# 크롬 드라이버를 사용해서 웹 브라우저 실행하기 

s_time = time.time( )

path = "/Users/kibeomkim/Desktop/chromedriver_240/chromedriver"
driver = webdriver.Chrome(path)

#1페이지로 이동
driver.get('https://www.naver.com/')
time.sleep(random.randrange(2,5))


#네이버 검색창 찾고 검색어 넣고 엔터
element1 = driver.find_element_by_id('query')
element1.send_keys(key_word)
element1.send_keys('\n')
time.sleep(2)



contents_list111 = ""

#검색옵션 선택하기 
try : 
    driver.find_element_by_class_name('bt_option').click()
    time.sleep(2)
except : 
    print("")

driver.find_element_by_link_text('기간').click()
time.sleep(2)
element2 = driver.find_element_by_id('_nx_date_from')
element2.send_keys(start_date)
element3 = driver.find_element_by_id('_nx_date_to')
element3.send_keys(end_date)

driver.find_element_by_class_name('tx').click()
time.sleep(2)

driver.find_element_by_class_name('m _tab_option_').click()
time.sleep(2)

element4 = driver.find_element_by_class_name('_input_include_')
element4.send_keys(include_word)

element5 = driver.find_element_by_class_name('_input_exclude_')
element5.send_keys(exclude_word)

driver.find_element_by_class_name('btn_search active _btn_search_').click()
time.sleep(2)

driver.find_element_by_link_text('블로그').click()
time.sleep(2)

url_list1 = []
nick_list1 = []
date_list1 = []
con_list1 = []

no2 = 2
#무한스크롤링 필요함. 

while no2 == 2 : 
    html2 = driver.page_source
    soup2 = BeautifulSoup(html2, 'html.parser')
    blog_list011 = soup2.find('ul', 'lst_total').find_all('li')
    
    if cnt > len(blog_list011) : 
        try : 
            driver.find_element_by_tag_name('ul').send_keys(Keys.END) # <-- 스크롤 다운 함수 자리
        except : 
            break
            print('요청하신 %s 건 대신 %s 건 만큼만 자료가 있어서 %s 건 만큼만 자료 수집하겠습니다' %(cnt, len(blog_list011), len(blog_list011))
            #cnt = len(blog_list011)    
    if cnt == len(blog_list011) : 
        break
    if cnt < len(blog_list011) : 
        break

                  
        
#블로그 url 리스트 만들기 
blog_url_list222 = []
html12 = driver.page_source
soup12 = BeautifulSoup(html0, 'html.parser')
contents_list12 = soup12.find('ul', 'lst_total').find_all('li', 'bx')
no3 = 1

for b in contents_list12 : 
    blog_url12 = b.find('a', 'api_txt_lines total_tit')['href']
    blog_url_list222.append(blog_url12)
    no3 += 1
    if no3 > cnt : 
        break
    #blog_url_list222 는 url 리스트이다. 

driver.close()
                  
f = open(ff_name, 'a', encoding='utf-8')  

#블로그 리스트로 하나하나 들어가서 상세_내용정보 모으기
for c in blog_url_list222 : 
    path = "/Users/kibeomkim/Desktop/chromedriver_240/chromedriver"
    driver = webdriver.Chrome(path)
    driver.get(c)

    #2-블로그 하나하나 들어가서 크롤링 시작
    html0 = driver.page_source
    soup0 = BeautifulSoup(html0, 'html.parser')

    #블로그 속에서 정보 얻기
    asdf = driver.switch_to.iframe('mainFrame')
    html900 = driver.page_source
    soup900 = BeautifulSoup(html900, 'html.parser')
                  
    f.write('총 %s 건 중 %s 번째 블로그 데이터를 수집합니다===========================' %(len(blog_list011), no1))
    f.write('\n')
                  
    f.write('1. 블로그 주소 :'+c)
    f.write('\n')
    url_list1.append(c)
    
    nick_name = soup900.find('a', 'link pcol2').get_text().replace('\n', "").strip()
    f.write('2. 작성자 :'+nick_name)
    f.write('\n')
    nick_list1.append(nick_name)
    
    date_info = soup900.find('span', 'se_publishDate pcol2').get_text().replace('\n', "").strip()
    f.write('3. 작성일자 :'+date_info)
    f.wrtie('\n')
    date_list1.append(date_info)
    
    contents_list2 = soup900.find_all('div', 'se-section se-section-text se-l-default') 
    for b in contents_list2 : 
        contents = b.get_text().replace('\n', "").strip()+" "
        contents_list111 += contents
    #contents_list111 1 블로그 내용 총합이다
    con_list1.append(contents_list111)         
    f.write('4. 블로그 내용 :'+contents_list111)
    f.write('\n')
    f.write('\n')
            
    driver.close()      
    
f.close()

e_time = time.time()
total_time = e_time - s_time 

naver = pd.DataFrame()
naver['블로그주소'] = url_list1 
naver['작성자닉네임'] = nick_list1
naver['작성일자'] = date_list1
naver['블로그내용'] = con_list1

naver.to_csv(fc_name, encoding='utf-8-sig')
naver.to_excel(fx_name)

                  
print('='*100)
print('요청하신 %s 건의 자료를 모두 수집하였습니다' %cnt)
print('데이터 수집에 걸린 소요시간은 %s 입니다' %total_time)
print('='*100)




SyntaxError: invalid syntax (<ipython-input-1-5bcb19e3d579>, line 129)

In [30]:
driver.find_element_by_tag_name('body').send_keys(Keys.END) 



NameError: name 'driver' is not defined

'dsds'